# Generate Reports

This notebook will read in the data for an Arable Mark, generate a suite of visual data (i.e. a "dashboard" for the Mark) and then export the data and accompanying text as a pdf file suitable for distribution to Mark hosts in Laikipia, Kenya.

The general workflow will be:

1. Import most recent data for an individual Mark using the Arable API.
1. Use the plotly library to create climate/vegetation data visualizations.
1. Export visualizations as .png files for inclusion in the report.
1. Use Jinja2 templates to create an html version of the report.
1. Use weasyprint to create a pdf from the html version of the report

Additional Goals:

1. Since we are making html versions of this report, let's setup a website that can auto-populate this information. The reports might be static, but perhaps through the magic of plotly/dash, we will be able to make them dynamic.

1. Put this site within an Arable app() context, so that this same general system could be used for any Arable domain. This will replicate a lot of the "functionality" of the existing JS-based website, but who cares. Web dev is fun.

## Import data using the Arable API.

First we need to setup the environment for our notebook. This requires some imports, etc... and we need to side load the Arable API library via a `!pip` command. There is no need to install Jinja2 because it comes with JupyterLab on Azure.


In [3]:
!pip install arable

  Stored in directory: /home/nbuser/.cache/pip/wheels/f6/77/7e/bd4d0e497cdf3fd5a85af5ba57e171b25ebdde30c372735415
Successfully built arable
You should consider upgrading via the 'pip install --upgrade pip' command.


In [4]:
from jinja2 import Template, Environment, FileSystemLoader, select_autoescape
PATH = '/home/nbuser/library'
env = Environment(
    loader=FileSystemLoader(PATH + '/mark_reports/templates', followlinks=True), # requires library to be added to path
    autoescape=select_autoescape(['html', 'xml'])
)

### Setup our credentials

_TODO: How to obscure the credentials in Azure?_


In [5]:
arable_email = 'caylor@ucsb.edu'
arable_password = 'greenresearch11'
arable_tenant = 'smallholder'
device_name = 'A000706'  # This is the device that we will build a report for. Will be passed eventually

from arable.client import *

a = ArableClient()
a.connect(arable_email, arable_password, arable_tenant)

d = a.devices(name=device_name)

### Let's get some info on this device.

At this point, we are going to start putting together material for our report. The device, `d` is just a dictionary, with the following keys: 

```python
['batt_pct', 'batt_volt', 'created', 'firmware', 'flags', 'id', 'last_deploy', 'last_post', 'last_seen', 'location', 'model', 'name', 'org', 'owner', 'permissions', 'reported_fw', 'roles', 'signal_strength', 'state', 'sync_interval', 'updated']
```

Timestamps in the Arable API look like `2019-06-25T09:21:35.626000`, so `TIME_FORMAT=%Y-%m-%DT%H:%M:%S.%f`

Also, we will define a couple of custom filters to handle elapsed times (timedeltas) and current time in more human-friendly formats.

In [7]:
import datetime
#from weasyprint import HTML
TIME_FORMAT="%Y-%m-%dT%H:%M:%S.%f"
def elapsed_time(value, format=TIME_FORMAT):
    # Uses the current time to determine elapsed time since value.
    # Returns the elapsed time in a human-friendly format string
    # (e.g. 3 weeks, or 1 day, 10 hours, and 21 minutes)
    import humanfriendly
    from datetime import timedelta, datetime
    return humanfriendly.format_timespan(
        (datetime.now()-datetime.strptime(value, format)).total_seconds()
    )

def format_time(value, format="%B %d, %Y"):
    # Format datetimes into "January 10, 2019" strings
    return value.strftime(format)

# Append these custom filters to the list of Jinja2 filters:
env.filters['elapsed_time'] = elapsed_time
env.filters['format_time'] = format_time

In [8]:
status_template = env.get_template('status.html')


status_html = status_template.render(
    date=datetime.datetime.now(),
    name=d['name'],
    voltage=d['batt_volt'],
    pct=d['batt_pct'],
    state=d['state'],
    strength=d['signal_strength'],
    updated=d['updated']
    )

with open("{name}_status.html".format(name=d['name']), "w") as fh:
    fh.write(status_html)

### Gather recent data for this device

Now that we have the device and basic information about its status, let's get some data
